Привет еще раз! меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на красные комментарии ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за очередную качественную работу, было интересно ее проверять.
- Я оставил некоторые советы надеюсь они будут полезными или интересными.
- Техника бутстрап применилась корректно, было проведено тщательное исследование и выбран правильный регион!
- Отправляю проект назад, чтобы у тебя была возможность задать вопросы, если они у тебя есть. Если их нет, то можешь просто отправить проект еще раз и я его зачту



<div class="alert alert-warning">
<b> Комментарий студента 2</b>      
    
Люман, привет! Спасибо тебе еще раз за ревью и ссылки по будстрапу, в спринте эта тема разобрана очень поверхностно, так что дополнительный материал очень кстати! Вопросов пока больше нет, спасибо еще раз!</div>


<div class="alert alert-block alert-success">
<b>Ревьювер[2]:</b> Было приятно с тобой сотрудничать. Не буду больше задерживать, продолжай в том же духе.
</div>

# Выбор локации для скважины

# Введение  
Добывающая компании «ГлавРосГосНефть», планирует разработку нового нефтяного месторождения. Поставлена задача определить регион для разработки. Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо разработать модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

# Общая информация о данных  
Данные геологоразведки трёх регионов находятся в файлах:   
/datasets/geo_data_0.csv. Скачать датасет  
/datasets/geo_data_1.csv. Скачать датасет  
/datasets/geo_data_2.csv. Скачать датасет  
id — уникальный идентификатор скважины;  
f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
product — объём запасов в скважине (тыс. баррелей).  

# Дополнительные условия задачи  
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

# План работы  

1. Импорт данных и библиотек
2. Исследовательский анализ данных  
3. Подготовка к расчету прибыли  
4. Расчет прибыли и рисков  
5. Общий вывод

## Импорт данных и библиотек

In [ ]:
#!pip3 install -U pandas-profiling

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc, \
precision_recall_curve
 
from sklearn.metrics import mean_squared_error

pd.options.display.max_columns = None

import scipy.stats as st
from scipy.stats import bootstrap

import pandas_profiling as pp
from pandas_profiling import ProfileReport

In [ ]:
# импорт данных
geo0 = pd.read_csv('/datasets/geo_data_0.csv')
geo1 = pd.read_csv('/datasets/geo_data_1.csv')
geo2 = pd.read_csv('/datasets/geo_data_2.csv')

## Исследовательский анализ данных

In [ ]:
# создадим список датасетов
all_sets = [geo0,geo1,geo2]
sets_names = ['geo0','geo1','geo2']

In [ ]:
# единое форматирование названий колонок 
for i in all_sets:
    i.columns = [x.lower().replace(' ', '_') for x in i.columns.values]
    display(i.head(1))

In [ ]:
#EDA при помощи библиотеки pandas_profiling для всех 3х датасетов
profile_0 = ProfileReport(geo0, 
                            title='geo0', 
                            explorative=True)#, 
                            #html={'style':{'full_width':True}})
profile_1 = ProfileReport(geo1, 
                            title='geo0', 
                            explorative=True)
profile_2 = ProfileReport(geo2, 
                            title='geo0', 
                            explorative=True)

In [ ]:
# вывод результатов EDA
profile_0

In [ ]:
profile_1

In [ ]:
profile_2

***Вывод***  

Для анализа предоставлены 3 датасета, каждый из которых включает 10000 наблюдений для 4 метрик: f0, f1, f2, product. Пропусков в данных не обнаружено. Явных дубликатов нет. В данных регионов, которым соответствуют датасеты geo0 и geo2 выявлена прямая умеренная корреляция метрики f2 и product. Для региона, которому соответствует датасет geo1 выявлена сильная прямая корреляция между метриками f2 и product.   
Средние показатели объемов запасов сырья в скважине составили:  
регион geo0 - 92 тыс. баррелей,  
регион geo0 - 69 тыс. баррелей,  
регион geo3 - 95 тыс. баррелей.


<div class="alert alert-block alert-success">
<b>Успех:</b> Данные осмотрены хорошо, использован интересный инструмент =)
</div>


<div class="alert alert-block alert-info">
<b>Совет:</b> Можно было бы еще выдвинуть предположение, откуда такая высокая корреляция во 2 регионе с целевым признаком могла возникнуть
</div>


## Подготовка к расчёту прибыли

In [ ]:
points = 500
target_points = 200
budget = 10**10
inc_per_bar = 450
inc_per_unit = 450000
rnd_st = np.random.RandomState(12345)

<div class="alert alert-block alert-info">
    
<b>Совет:</b> Константы принято записывать в ВЕРХНЕМ РЕГИСТРЕ =) Читем здесь https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html#section-29
    
А большие значение можно записывать так: `450_000`.
</div> 

In [ ]:
# удалим параметр id из датасетов, т.к. он не будет использован для расчетов
geo0 = geo0.drop(['id'], axis=1)
geo1 = geo1.drop(['id'], axis=1)
geo2 = geo2.drop(['id'], axis=1)

In [ ]:
vol_of_raw = budget/target_points/inc_per_unit
display(f'необходимый объём сырья для безубыточной разработки новой скважины:{round(vol_of_raw,3)}')

***Вывод***  
Для безубыточной разработки новой скважины, требуется получение из нее 111.111 тыс. баррелей сырья.


<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен  
</div>


## Расчёт прибыли и рисков 

In [ ]:
#определяем функцию, разделяющую данные на обучающую и тестовую выборки,
#обучающую модель и возвращающую метрики качества rmse и r2 и прогнозируемый параметр product
def split_and_train(data):
 
    features = data.drop("product", axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=12345)
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    lr = LinearRegression(normalize=False)
    lr.fit(features_train, target_train)
    predictions = lr.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = (mean_squared_error(predictions, target_valid))**(0.5)
    r2 = lr.score(features_train,target_train)
    average_product = sum(predictions) / len(predictions)
    
    print(f"Среднее количество прогнозируемого сырья: \
    {round(average_product,3)}, rmse: {round(rmse,3)}, r2: {round(r2,3)}")
    
    return (predictions, target_valid.reset_index(drop=True), rmse, average_product)

In [ ]:
predictions_1, target_1, rmse_1, avg_1 = split_and_train(geo0)
predictions_2, target_2, rmse_2, avg_2 = split_and_train(geo1)
predictions_3, target_3, rmse_3, avg_3 = split_and_train(geo2)

<div class="alert alert-block alert-success">
<b>Успех:</b> Шаг проделан лаконично, все метрики посчитаны корректно
</div> 


In [ ]:
# функция для отбора скважин и последующего расчета прибыли 
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]#count] # выбор скважин с максимальным прогнозом модели
    return inc_per_unit * selected.sum() - budget



<div class="alert alert-block alert-success">
<b>Успех:</b> Ф-ция для подсчета прибыли определена верно
</div>

In [ ]:
# функция для расчета прибыли и рисков 
region = ['geo0', 'geo1', 'geo2']
def risk(tgt, pred, num):
    values = []
    for j in range(1000):
        target_subsample = tgt.sample(n = 500, replace = True, random_state = rnd_st)
        probs_subsample = pred.loc[target_subsample.index]

        values.append(revenue(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = values[values < 0].count()/values.count()*100
    mean = values.mean()
    print(region[num])
    print("Средняя прибыль:", round(mean/10**6, 2), 'млн.руб')
    print("Доверительный интервал от:", round(lower/10**6, 2),'млн.руб.', 'до:', round(upper/10**6, 2),'млн.руб')
    print("Риск:", round(risk, 2), '%')

<div class="alert alert-block alert-success">
<b>Успех:</b> С бутстрапом тоже все в порядке, метрики считаются верным образом!  
</div>

In [ ]:
risk(target_1, predictions_1, 0)
risk(target_2, predictions_2, 1)
risk(target_3, predictions_3, 2)


<div class="alert alert-warning">
<b> Комментарий студента</b>      
    
Пытался решить задачу через пакет bootstrap из scipy. Если в функции revenue вместо count напрямую задать 200, и убрать третий аргумент функции, то казалось бы, все должно работать. Но код ниже выдает ошибку вроде "np.array has not method sort_values " или что-то в том духе. Буду благодарен за совет как это можно было бы решить.</div>


<div class="alert alert-block alert-info">
<b>Совет:</b> 
    
 Я ранее не использовал функцию bootstrap из scipy, но твоя ошибка просто связана с тем, что sort_values команда библиотеки пандас - для датафрейма и серий. А boostrap из scipy на вход функции revenue передает объекты в виде numpy - решение простое, оптимизировать функцию прибыли под этот кейс и преобразовать данные в серию. НО сделав это я все рвано не смог запустить код, так как у меня падало ядро, что на платформе, что локально.  Слишком ресурсоемкий метод получился.
</div>


In [ ]:
# доверительные интервалы для roc-auc, определенные bootstra'ом
'''boot_res = bootstrap((target_1, predictions_1), 
                     revenue, 
                     vectorized=False,
                     paired=True,
                     n_resamples=1001, 
                     random_state=12345)

print(boot_res.confidence_interval)'''


<div class="alert alert-block alert-info">
<b>Совет:</b> Интересно при чем здесь roc_auc = )
</div>


<div class="alert alert-warning">
<b> Комментарий студента 2</b>      
    
о, случайно остался комментарий, параллельно пытаюсь разобраться в статистике, и для расчета ДИ эта функция работала))</div>

## Общий вывод  

1. Выполнен исследовательский анализ данных, по результатам которого установлено:  
Каждый анализируемый датасет включает 10000 наблюдений для 4 метрик: f0, f1, f2, product. Пропусков в данных не обнаружено. Явных дубликатов нет. В данных регионов, которым соответствуют датасеты geo0 и geo2 выявлена прямая умеренная корреляция метрики f2 и product. Для региона, которому соответствует датасет geo1 выявлена сильная прямая корреляция между метриками f2 и product.
Средние показатели объемов запасов сырья в скважине составили:
регион geo0 - 92 тыс. баррелей,
регион geo0 - 69 тыс. баррелей,
регион geo3 - 95 тыс. баррелей.  

2. Установлено, что для безубыточной разработки новой скважины, требуется получение из нее 111.111 тыс. баррелей сырья.  

3. При прогнозе прибыли выявлено, что регион добычи, соответствующий датасету "geo1", обладает наименьшим риском - 0,3%, что соответствует требованию заказчика, ожидаемая прибыль составляет 518.26 млн. руб. Согласно проведенному анализу данный регион должен иметь приоритет при дальнейшей разработке месторождений.

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, проведено детальное исследование и был выбран правильный регион!
</div>



<div class="alert alert-block alert-info">
<b>Совет: </b> Если интересно изучить тему бутстрапа глубже - я оставлю  ниже несколько интересных и полезных ссылок по теме:
        
1. "Bootstrap Hypothesis Testing in Statistics with Example" 
        
https://www.youtube.com/watch?v=9STZ7MxkNVg
        
2. "How to Calculate Bootstrap Confidence Intervals For Machine Learning Results in Python" 
        
https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/
        
3. "A Gentle Introduction to the Bootstrap Method" 

https://machinelearningmastery.com/a-gentle-introduction-to-the-bootstrap-method/
        
4. "An Introduction to the Bootstrap Method" 
        
https://towardsdatascience.com/an-introduction-to-the-bootstrap-method-58bcb51b4d60#:~:text=The%20basic%20idea%20of%20bootstrap,population%20mean)%20on%20sample%20data.&amp;text=A%20sample%20from%20population%20with%20sample%20size%20n
        
5. "Bootstrapping and Resampling in Statistics with Example" 
        
        
https://www.youtube.com/watch?v=O_Fj4q8lgmc

6. Также одна из программ на рынке по этой теме управлненеческого моделирования - AnyLogic - интересный продукт. Можешь видео из интернета посмотреть по какому-нибудь примеру с её использованием. Тогда тема управленческого моделирования и бутстрапа может по-другому лечь.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован